In [1]:
import torch
from torch import nn
import pcn_kernels

class pcnpass(torch.autograd.Function):
    @staticmethod
    def forward(c: torch.Tensor, l: torch.Tensor, lnext: torch.Tensor, b: torch.Tensor):
        output = pcn_kernels.forward(c, l, lnext, b)
        return output
    
    @staticmethod
    def setup_context(ctx, inputs, _):
        c, l, lnext, b = inputs
        ctx.save_for_backward(c, l, lnext, b)

    @staticmethod
    def backward(ctx, grad_cnext):
        c, l, lnext, b = ctx.saved_tensors
        grad_c, grad_l, grad_lnext, grad_b = pcn_kernels.backward(grad_cnext.contiguous(), c, l, lnext, b)
        return grad_c, grad_l, grad_lnext, grad_b


class PCN(nn.Module):
    def __init__(
        self,
        layers: list[int],
        dimensions: int = 20,
    ):
        super().__init__()
        if len(layers) < 2:
            raise ValueError("At least 2 layers are required")

        self.layers = nn.ParameterList(
            [nn.Parameter(torch.rand(l, dimensions) * 2 - 1) for l in layers]
        )
        self.layers_bias = nn.ParameterList(
            [nn.Parameter((torch.rand(l) * 2 - 1) * 0.1) for l in layers]
        )

    def forward(self, x: torch.Tensor):
        z = x
        for i, (l, lnext) in enumerate(zip(self.layers, self.layers[1:])):
            z = pcnpass.apply(z, l, lnext, self.layers_bias[i + 1])
            if i < len(self.layers) - 2:
                z = torch.relu(z)
        return z

# control model
class FCN(nn.Module):
    def __init__(
        self,
        n_in: int,
        n_out: int,
        hidden: list[int],
    ):
        super().__init__()

        c = n_in
        L = []
        for l in hidden:
            L.append(nn.Linear(c, l))
            L.append(nn.ReLU())
            c = l

        L.append(nn.Linear(l, n_out))
        self.net = nn.Sequential(*L)

    def forward(self, x):
        return self.net(x)

In [2]:
gpu = torch.device("cuda:0")

in_size = 1000
out_size = 100

num_tests = 100
x = torch.rand(num_tests, in_size).to(gpu)
pcn = PCN([in_size, out_size]).to(gpu)
pcn(x)

tensor([[ 0.3311,  0.1757, -0.4517,  ...,  0.0432,  0.0869, -0.0188],
        [ 0.1551,  0.1073, -0.0605,  ..., -0.2180,  0.1503,  0.0268],
        [ 0.0686,  0.2837, -0.3161,  ...,  0.3544,  0.0527,  0.0246],
        ...,
        [ 0.1341, -0.0568, -0.3880,  ...,  0.1064,  0.0698, -0.1622],
        [ 0.0499,  0.1083, -0.0751,  ...,  0.0617,  0.4562, -0.4796],
        [ 0.1600,  0.2338, -0.4881,  ..., -0.0972, -0.2094, -0.1343]],
       device='cuda:0', grad_fn=<pcnpassBackward>)

In [3]:

# {layer size} -> {output variance} analysis


in_size = 1000
out_size = 100

num_tests = 100
agg_var = []
agg_mean = []
for i in range(num_tests):
    pcn = PCN([in_size, in_size, in_size, in_size, out_size]).to(gpu)
    agg_var.append(pcn(torch.rand(1, in_size).to(gpu)).var())
    agg_mean.append(pcn(torch.rand(1, in_size).to(gpu)).mean())

torch.stack(agg_mean).mean(), torch.stack(agg_var).mean()


(tensor(-5.5202e-05, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0045, device='cuda:0', grad_fn=<MeanBackward0>))